In [ ]:
#!./cleaner.py

In [5]:
import pandas as pd
import spacy,random

In [2]:

jd_data=pd.DataFrame(columns=['Job Description','Cleaned JD'])
type_names=['DataAnalyst','DataScientist','BusinessAnalyst','DataEngineer']
for each in type_names:
    try:
        jd=pd.read_csv("data/cleaned/{}.csv".format(each))
        jd_data=pd.concat([jd_data,jd[jd_data.columns]],ignore_index=True)
    except:
        pass

In [3]:
jds=jd_data['Job Description'].dropna().reset_index(drop=True)
jd_cleaned=jd_data['Cleaned JD'].dropna().reset_index(drop=True)

jd_useful=[]
jd_all=[]
jd_waste=[]
for i in range(len(jd_cleaned)):
    jd_useful.extend(list(filter(None,jd_cleaned[i].splitlines())))
for i in range(len(jd_cleaned)):
    jd_all.extend(list(filter(None,jds[i].splitlines())))
for i in jd_all:
    if i not in jd_useful:
        jd_waste.append(i)


In [6]:
nlp=spacy.blank('en')
docs=[]
for i in jd_useful:
    doc=nlp(i)
    doc.cats['LABEL']=1
    doc.cats['NOT_LABEL']=0
    docs.append(doc)
for i in jd_waste:
    doc=nlp(i)
    doc.cats['LABEL']=0
    doc.cats['NOT_LABEL']=1
    docs.append(doc)

random.shuffle(docs)
train_docs=docs[:len(docs)//2]
dev_docs=docs[len(docs)//2:]

spacy.tokens.DocBin(docs=train_docs).to_disk("./clean_model/corpus/train.spacy")
spacy.tokens.DocBin(docs=dev_docs).to_disk("./clean_model/corpus/dev.spacy")

In [7]:
len(docs)

1701

In [11]:
# redirect into model dir
# python -m spacy train ./config.cfg --output ./output


In [8]:
nlp=spacy.load("./clean_model/output/model-best")
nlp.pipe_names

['textcat']

In [14]:
def clean_by_model(line):
    str=''
    jd_i=list(filter(None,line.splitlines()))
    for i in jd_i:
        if (nlp(i).cats['LABEL']>0.5):
            str=str+i+'\r\n'
    return str

for each in type_names:
    jd_data=pd.read_csv("data/origin/{}.csv".format(each))
    jd_data['Cleaned JD']=jd_data['Job Description'].apply(clean_by_model)
    jd_data.to_csv("data/generated/{}.csv".format(each))

In [ ]:
import spacy
nlp = spacy.load('en_core_web_md')
nlp.pipe_names